In [1]:
from pyspark.sql import SparkSession
import os
import subprocess
import json
import pyspark.sql.functions as psf
spark = SparkSession.builder.getOrCreate()

Picked up JAVA_TOOL_OPTIONS: -XX:+UseContainerSupport -XX:ActiveProcessorCount=1
Picked up JAVA_TOOL_OPTIONS: -XX:+UseContainerSupport -XX:ActiveProcessorCount=1
24/09/12 16:16:11 WARN Utils: Your hostname, thomasnelis-capstonellm-e22sbqjnblg resolves to a loopback address: 127.0.0.1; using 10.0.5.2 instead (on interface eth0)
24/09/12 16:16:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/12 16:16:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
tag = 'airflow'
questions_in = spark.read.json(f"s3://dataminded-academy-capstone-llm-data-us/input/{tag}/questions.json")

24/09/12 16:19:05 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3://dataminded-academy-capstone-llm-data-us/input/airflow/questions.json.
org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:366)
	at org.apache.spark.sql.DataFrameReade

Py4JJavaError: An error occurred while calling o31.json.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNecessary(Dat

In [15]:
if not os.path.isfile("./data_in/airflow/questions.json"):
    subprocess.run(["aws", "s3", "cp", "s3://dataminded-academy-capstone-llm-data-us/input/airflow/questions.json", "./data_in/airflow/"])
if not os.path.isfile("./data_in/airflow/answers.json"):
    subprocess.run(["aws", "s3", "cp", "s3://dataminded-academy-capstone-llm-data-us/input/airflow/answers.json", "./data_in/airflow/"])

In [16]:
questions_obj = spark.read.json("./data_in/airflow/questions.json")
answers_obj = spark.read.json("./data_in/airflow/answers.json")

In [17]:
questions = ( 
    questions_obj
        .select(psf.explode(questions_obj.items).alias("items"))
        .filter(psf.col("items.accepted_answer_id").isNotNull())
        .select(psf.col("items.body").alias("question"), "items.title", "items.is_answered", "items.accepted_answer_id")
)
print(questions.count())

62


In [18]:
answers = ( 
    answers_obj
        .select(psf.explode(answers_obj.items).alias("items"))
        .select(psf.col("items.body").alias("answer"), "items.answer_id", "items.question_id")
)
print(answers.count())

30


In [19]:
output = questions.join(answers, on=(questions.accepted_answer_id == answers.answer_id)).select("title", "question", "answer")
print(output.count())
output.show()

21
+--------------------+--------------------+--------------------+
|               title|            question|              answer|
+--------------------+--------------------+--------------------+
|Airflow: how to d...|<p>I have started...|<p><strong>Edit 8...|
|execution_date in...|<p>I am really a ...|<p>The <code>Bash...|
|Apache Airflow or...|<p>I'm trying to ...|<p>The other answ...|
|How to create a c...|<p>I would like t...|<p>You have to us...|
|How to prevent ai...|<p>Say you have a...|<p>Upgrade to air...|
|First time login ...|<p>I've just inst...|<p>There is no de...|
|How to control th...|<p>In some of my ...|<p>Here's an expa...|
|Writing to Airflo...|<p>One way to wri...|<p>You can import...|
|Airflow - How to ...|<p>I need to refe...|<p>Templates like...|
|Airflow s3 connec...|<p>I've been tryi...|<p>EDIT: This ans...|
|Airflow versus AW...|<p>I am working o...|<p>I have worked ...|
|Error while insta...|<p>Getting the fo...|<p>Try the follow...|
|How to restart a ...|

In [20]:
output.repartition(output.count()).write.mode("overwrite").json("./data_out/")

In [22]:
for filename in os.listdir("./data_out/airflow/"):
    if not filename.endswith(".json"):
        os.remove(os.path.join("./data_out/airflow/", filename))

FileNotFoundError: [Errno 2] No such file or directory: './data_out/airflow/'

In [13]:
for filename in os.listdir("./data_out"):
    f = os.path.join("./data_out", filename)
    if os.path.isfile(f) and filename.split('.').pop() == 'json':
        print("uploading " + filename)
        subprocess.run(["aws", "s3", "cp", "./data_in/airflow/"+filename, "s3://dataminded-academy-capstone-llm-data-us/cleaned/thomas/airflow/"])

uploading part-00000-5bf065cc-ea5d-4b28-8b1c-3522234ac00b-c000.json



The user-provided path ./data_in/airflow/part-00000-5bf065cc-ea5d-4b28-8b1c-3522234ac00b-c000.json does not exist.


uploading part-00001-5bf065cc-ea5d-4b28-8b1c-3522234ac00b-c000.json



The user-provided path ./data_in/airflow/part-00001-5bf065cc-ea5d-4b28-8b1c-3522234ac00b-c000.json does not exist.


uploading part-00002-5bf065cc-ea5d-4b28-8b1c-3522234ac00b-c000.json


...


The user-provided path ./data_in/airflow/part-00018-5bf065cc-ea5d-4b28-8b1c-3522234ac00b-c000.json does not exist.


uploading part-00019-5bf065cc-ea5d-4b28-8b1c-3522234ac00b-c000.json



The user-provided path ./data_in/airflow/part-00019-5bf065cc-ea5d-4b28-8b1c-3522234ac00b-c000.json does not exist.


uploading part-00020-5bf065cc-ea5d-4b28-8b1c-3522234ac00b-c000.json



The user-provided path ./data_in/airflow/part-00020-5bf065cc-ea5d-4b28-8b1c-3522234ac00b-c000.json does not exist.


In [4]:
import requests
import json

response = requests.get("https://api.stackexchange.com/2.3/questions?site=stackoverflow")
print(json.loads(response.text))

{'items': [{'tags': ['ios', 'swift', 'uitableview', 'uikit', 'uistackview'], 'owner': {'account_id': 1459679, 'reputation': 4810, 'user_id': 1374327, 'user_type': 'registered', 'accept_rate': 95, 'profile_image': 'https://i.sstatic.net/2x0LX.jpg?s=256', 'display_name': 'NSPratik', 'link': 'https://stackoverflow.com/users/1374327/nspratik'}, 'is_answered': False, 'view_count': 1, 'answer_count': 0, 'score': 0, 'last_activity_date': 1726217628, 'creation_date': 1726217628, 'question_id': 78981401, 'content_license': 'CC BY-SA 4.0', 'link': 'https://stackoverflow.com/questions/78981401/how-to-dynamically-add-rows-to-uitableviewcell-in-efficient-manner', 'title': 'How to dynamically add rows to UITableviewCell in efficient manner?'}, {'tags': ['ios', 'swift', 'flutter', 'swiftui', 'mobile'], 'owner': {'account_id': 20433210, 'reputation': 33, 'user_id': 14991711, 'user_type': 'registered', 'profile_image': 'https://lh3.googleusercontent.com/a-/AOh14Gi5H4cyU7JbhszeeAXyUFdaMEtMbV99P0mquzfg=k